In [123]:
# TensorFlow and TF-Hub modules.
from absl import logging

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

from collections import defaultdict


logging.set_verbosity(logging.ERROR)

# Some modules to help with reading the dataset.
import pandas as pd
import random
import re
import os
import shutil
import tempfile
import ssl
import cv2
import numpy as np

# Some modules to display an animation using imageio.
import imageio
from IPython import display

from urllib import request  # requires python3

In [72]:
# 2D Convolutional Neural Network.
def cnn_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu',
                        input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))    
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [73]:
# Train the model.
def train_model(model, epochs=5):

    # Create the image generators
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
    
    val_datagen = ImageDataGenerator(rescale=1./255)
    
    # Create the image generators
    train_generator = train_datagen.flow_from_directory(
        'sample_set/',  
        target_size=(150, 150),
        classes=['real', 'fake'],
        batch_size=32,
        class_mode='binary')
    
    val_generator = val_datagen.flow_from_directory(
        'sample_set/',  
        target_size=(150, 150), 
        classes=['real', 'fake'],
        batch_size=32,
        class_mode='binary')
    
    # Train the model
    model.compile(loss='binary_crossentropy',
                    optimizer=Adam(lr=0.001),
                    metrics=['accuracy'])
    model.fit(
        train_generator,
        steps_per_epoch=25,
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=10) 
    
    # Save the model
    model.save("model.h5")
    # Return the model
    return model

In [75]:
model   = cnn_model((150, 150, 3))
model   = train_model(model)

Found 1014 images belonging to 2 classes.
Found 1014 images belonging to 2 classes.
Epoch 1/5
25/25 [==============================] - 14s 517ms/step - loss: 0.5572 - accuracy: 0.7823 - val_loss: 0.2686 - val_accuracy: 0.8781
Epoch 2/5
25/25 [==============================] - 13s 512ms/step - loss: 0.0782 - accuracy: 0.9800 - val_loss: 0.0300 - val_accuracy: 0.9875
Epoch 3/5
25/25 [==============================] - 13s 508ms/step - loss: 0.0703 - accuracy: 0.9823 - val_loss: 0.0049 - val_accuracy: 1.0000
Epoch 4/5
25/25 [==============================] - 13s 498ms/step - loss: 0.0431 - accuracy: 0.9886 - val_loss: 0.0013 - val_accuracy: 1.0000
Epoch 5/5
25/25 [==============================] - 13s 500ms/step - loss: 0.0344 - accuracy: 0.9924 - val_loss: 0.0077 - val_accuracy: 1.0000


In [119]:
class VideoCallback(keras.callbacks.Callback):
    def __init__(self, x):
        self.filepaths = x.filepaths

In [120]:
test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_directory(
    'sample_set/',  
    target_size=(150, 150),
    classes=['real', 'fake'],
    batch_size=32,
    class_mode='binary')

call = VideoCallback(test_generator)

pred = model.predict(test_generator, callbacks=[call])


Found 1014 images belonging to 2 classes.


In [131]:
vidoes = defaultdict(list)
for i, path in enumerate(call.filepaths):
    vid = path.split("\\")[1]
    vidoes[vid].append(i)


predictions = {}
for vid in vidoes:
    predictions[vid] = np.mean(pred[vidoes[vid]])
    
predictions
        

{'eggbjzxnmg.mp4': 0.5469747,
 'eudeqjhdfd.mp4': 0.5826362,
 'etohcvnzbj.mp4': 0.5760956,
 'eukvucdetx.mp4': 0.62330043}